In [1]:
import parseAndGigrnd
import random
import numpy as np
from scipy import linalg 
from numpy import random
import pandas as pd
import mcmc_gtb

In [2]:
# LD Panel References
references = pd.read_table('LD_Reference_Panel_Files\snpinfo_1kg_hm3')

SSTbim = pd.read_table('GWAS_Summary_Stats_Files\\test.bim', header = None, names = ['CHROM', 'SNP', '0', 'BP', 'A1', 'A2'])

# Summary Statistics
summaryStats = pd.merge(pd.merge(SSTbim, references, on='SNP', how='inner'), pd.read_table('GWAS_Summary_Stats_Files\sumstats_se.txt'), on='SNP', how='inner')
summaryStats['BETA'], summaryStats['FLP'] = np.where(SSTbim['A1'] == summaryStats['A1'], 1 * (summaryStats['BETA']) / summaryStats['SE'] / np.sqrt(200000), -1 * (summaryStats['BETA']) / summaryStats['SE'] / np.sqrt(200000)), 1
summaryStats['FLP'] = np.where(summaryStats['A1_x'] == summaryStats['A1_y'], 1 * summaryStats['FLP'], -1 * summaryStats['FLP'])
del summaryStats['A1'], summaryStats['A2']
summaryStats.rename(columns={"BP_x": "BP", "A1_x": "A1", "A2_x": "A2"}, inplace=True)

# LD Blocks and Block Sizes
LDBlock, blockSize = parseAndGigrnd.parse_ldblk('LD_Reference_Panel_Files', summaryStats, 22)

begin MCMC

In [3]:
param_dict = {'ref_dir': None, 'bim_prefix': None, 'sst_file': None, 'a': 1, 'b': 0.5, 'phi': None, 'n_gwas': None,
                'n_iter': 1000, 'n_burnin': 500, 'thin': 5, 'out_dir': None, 'chrom': range(1,23),
                'standardizedBeta': 'FALSE', 'write_psi': 'FALSE', 'write_pst': 'TRUE', 'seed': None}

mcmc_gtb.mcmc(param_dict['a'], param_dict['b'], 1e-2, summaryStats, 200000, LDBlock, blockSize,
    1000, 500, 5, 22, 'output', param_dict['standardizedBeta'], param_dict['write_psi'], param_dict['write_pst'], 5)

... MCMC ...
--- iter-100 ---
--- iter-200 ---
--- iter-300 ---
--- iter-400 ---
--- iter-500 ---
--- iter-600 ---
--- iter-700 ---
--- iter-800 ---
--- iter-900 ---
--- iter-1000 ---
... Done ...


In [7]:
positiveDefiniteMatrix = np.array([[1, 0.5], [0.5, 1]])

test = np.linalg.cholesky(positiveDefiniteMatrix)

np.dot(test, test.T)

array([[1. , 0.5],
       [0.5, 1. ]])

In [4]:
# Create a symmetric positive-definite matrix A
A = np.array([[4, 2, 2],
              [2, 6, 2],
              [2, 2, 5]])

# Ensure it's symmetric
A = (A + A.T) / 2

# Define the vector b
b = np.array([1, 2, 3])

# Perform Cholesky decomposition (A = LL^T)
L = np.linalg.cholesky(A)

# Print the lower triangular matrix L
print("Cholesky factor L:\n", L)

# Solve Ly = b using forward substitution
y = np.linalg.solve(L, b)

# Solve L^T x = y using backward substitution
x = np.linalg.solve(L.T, y)

# Print the solution x to Ax = b
print("Solution x:\n", x)

# Verify that Ax = b
print("Check Ax:\n", np.dot(A, x))


print(np.linalg.solve(A, b))

Cholesky factor L:
 [[2.         0.         0.        ]
 [1.         2.23606798 0.        ]
 [1.         0.4472136  1.94935887]]
Solution x:
 [-0.2814003   0.3881966   0.84477059]
Check Ax:
 [1.34033316 3.45592017 4.43744552]
[-0.13157895  0.18421053  0.57894737]
